## Setup

In [60]:
# check GPU
!pip install gpustat
!gpustat -i 1

774c5852b68148             Fri Dec 13 20:31:10 2019  418.67
[0] Tesla P100-PCIE-16GB | 43'C,   0 % |    10 / 16280 MB |
874c5852b68148             Fri Dec 13 20:31:11 2019  418.67
[0] Tesla P100-PCIE-16GB | 43'C,   0 % |    10 / 16280 MB |
874c5852b68148             Fri Dec 13 20:31:12 2019  418.67
[0] Tesla P100-PCIE-16GB | 43'C,   0 % |    10 / 16280 MB |
874c5852b68148             Fri Dec 13 20:31:13 2019  418.67
[0] Tesla P100-PCIE-16GB | 43'C,   0 % |    10 / 16280 MB |
874c5852b68148             Fri Dec 13 20:31:14 2019  418.67
[0] Tesla P100-PCIE-16GB | 43'C,   0 % |    10 / 16280 MB |
874c5852b68148             Fri Dec 13 20:31:15 2019  418.67
[0] Tesla P100-PCIE-16GB | 43'C,   0 % |    10 / 16280 MB |
88

In [0]:
from IPython.core.interactiveshell import InteractiveShell

# pretty print all cell's output and not just the last one
InteractiveShell.ast_node_interactivity = "all"

In [2]:
!curl -s https://course.fast.ai/setup/colab | bash
!git clone https://github.com/SpellOnYou/dlff-note.git
import os
os.chdir('/content/dlff-note/nbs/dl2/')

Updating fastai...
Done.
Cloning into 'dlff-note'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 5332 (delta 20), reused 37 (delta 16), pack-reused 5282
Receiving objects: 100% (5332/5332), 240.20 MiB | 50.74 MiB/s, done.
Resolving deltas: 100% (2881/2881), done.


In [0]:
%%script false
pip freeze

In [0]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## The forward and backward passes

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=4960)


In [0]:
#export
from exp.nb_01 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [6]:
x_train,y_train,x_valid,y_valid = get_data()

In [7]:
#getting mean and std
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.1304), tensor(0.3073))

In [0]:
# apply it to train and valid
x_train = normalize(x_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [9]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(3.0614e-05), tensor(1.))

In [0]:
#export
def test_near_zero(a,tol=1e-3): assert a.abs()<tol, f"Near zero: {a}"

In [0]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [12]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

## Foundations version

### Basic architecture

In [0]:
nh = 50
def lin(x, w, b): return x@w + b;
w1 = torch.randn(m,nh)*math.sqrt(2./m ); b1 = torch.zeros(nh)
w2 = torch.randn(nh,1); b2 = torch.zeros(1)

def relu(x): return x.clamp_min(0.) - 0.5
t1 = relu(lin(x_valid, w1, b1))

def model(xb):
    l1 = lin(xb, w1, b1) # one linear layer
    l2 = relu(l1) # one relu layer
    l3 = lin(l2, w2, b2) # one more linear layer
    return l3

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=5128)

In [0]:
# num hidden
nh = 50

[Thinker practice](https://course.fast.ai/videos/?lesson=8&t=5255)

##### Just grap some normal random numbers with linear function

In [0]:
def lin(x, w, b): return x@w + b

In [0]:
# If we do not divide by sqrt
w1 = torch.randn(m,nh)
b1 = torch.zeros(nh)
t = lin(x_valid, w1, b1) # hidden
t.mean(), t.std()

(tensor(2.4355), tensor(28.4015))

In [0]:
w2 = torch.randn(nh,1)
b2 = torch.zeros(1)
t2 = lin(t, w2, b2) # output
t2.mean(), t2.std()

(tensor(22.4911), tensor(198.5335))

In [0]:
t.shape, t2.shape

(torch.Size([10000, 50]), torch.Size([10000, 1]))

##### Kaiming Initialization

In [0]:
# simplified kaiming init / he init
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [0]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-1/math.sqrt(m))

In [0]:
# This should be ~ (0,1) (mean,std)...
x_valid.mean(),x_valid.std()

(tensor(-0.0057), tensor(0.9924))

In [0]:
t = lin(x_valid, w1, b1)

In [0]:
#...so should this, because we used kaiming init, which is designed to do this
t.mean(),t.std()

(tensor(0.1555), tensor(1.0225))

In [0]:
def relu(x): return x.clamp_min(0.) 

In [0]:
t = relu(lin(x_valid, w1, b1))

In [0]:
#...actually it really should be this!
t.mean(),t.std()

(tensor(0.4855), tensor(0.6525))

From pytorch docs: `a: the negative slope of the rectifier used after this layer (0 for ReLU by default)`

$$\text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan_in}}}$$

This was introduced in the paper that described the Imagenet-winning approach from *He et al*: [Delving Deep into Rectifiers](https://arxiv.org/abs/1502.01852), which was also the first paper that claimed "super-human performance" on Imagenet (and, most importantly, it introduced resnets!)

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=5128)

In [0]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2/m)
w1.mean(),w1.std()

t = relu(lin(x_valid, w1, b1))
t.mean(),t.std()

(tensor(0.4889), tensor(0.7556))

##### Torch package

In [0]:
#export
from torch.nn import init

In [0]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1, mode='fan_out') # why fan_out??
t = relu(lin(x_valid, w1, b1))

In [0]:
# init.kaiming_normal_??
help(init.kaiming_normal_)

Help on function kaiming_normal_ in module torch.nn.init:

kaiming_normal_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu')
    Fills the input `Tensor` with values according to the method
    described in `Delving deep into rectifiers: Surpassing human-level
    performance on ImageNet classification` - He, K. et al. (2015), using a
    normal distribution. The resulting tensor will have values sampled from
    :math:`\mathcal{N}(0, \text{std}^2)` where
    
    .. math::
        \text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan\_in}}}
    
    Also known as He initialization.
    
    Args:
        tensor: an n-dimensional `torch.Tensor`
        a: the negative slope of the rectifier used after this layer (0 for ReLU
            by default)
        mode: either ``'fan_in'`` (default) or ``'fan_out'``. Choosing ``'fan_in'``
            preserves the magnitude of the variance of the weights in the
            forward pass. Choosing ``'fan_out'`` preserves the magnitudes in t

See the 
- choosing 'fan_in' preserves the magnitude of the variance of the wights in the forward pass
- choosing 'fan_out' preserves the magnitues in the backward pass

In [0]:
w1.shape

torch.Size([784, 50])

In [0]:
import torch.nn

In [0]:
torch.nn.Linear(m,nh).weight.shape

torch.Size([50, 784])

**We should do fan_out(backward); cz pytorch use with transposed matrix**

In [0]:
# torch.nn.Linear.forward?? #what is this? why jeremy input this?
# help(torch.nn.Linear.forward)

In [0]:
%%script false
torch.nn.functional.linear??

In [0]:
%%script false
help(torch.nn.functional.linear)

In [0]:
t.mean(),t.std() #adequate to linear layer's variate, which is Relu.

(tensor(0.5668), tensor(0.8150))

In [0]:
torch.nn.Conv2d??

In [0]:
%%script false
torch.nn.Conv2d.conv2d_forward?? #jiwon put here

In [0]:
torch.nn.modules.conv._ConvNd.reset_parameters??

In [0]:
def relu(x): return x.clamp_min(0.)

In [0]:
# what if...? mitigate mean bias...

def relu(x): return x.clamp_min(0.) - 0.5

In [0]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2./m )
t1 = relu(lin(x_valid, w1, b1))
t1.mean(),t1.std()

(tensor(0.2070), tensor(0.9178))

Does it really enhance bias when we extract 0.5?? (just wanted to test jeremy)

WE can forward pass now!!!

1.   ReLU
2.   Linear
3.   Init





a model just can be a function

In [0]:
w1 = torch.randn(m,nh)*math.sqrt(2./m )
t1 = relu(lin(x_valid, w1, b1))
t1.mean(),t1.std()
def model(xb):
    l1 = lin(xb, w1, b1) # one linear layer
    l2 = relu(l1) # one relu layer
    l3 = lin(l2, w2, b2) # one more linear layer
    return l3

In [0]:
%timeit -n 10 _=model(x_valid)

10 loops, best of 3: 16.7 ms per loop


In [0]:
assert model(x_valid).shape==torch.Size([x_valid.shape[0],1]) #(10000, 1)

### Loss function: MSE

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=6372)

In [15]:
model(x_valid).shape

torch.Size([10000, 1])

We need `squeeze()` to get rid of that trailing (,1), in order to use `mse`. (Of course, `mse` is not a suitable loss function for multi-class classification; we'll use a better loss function soon. We'll use `mse` for now to keep things simple.)

In [0]:
#export
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

In [0]:
y_train,y_valid = y_train.float(),y_valid.float()

In [0]:
preds = model(x_train)

In [17]:
preds.shape

torch.Size([50000, 1])

In [21]:
mse(preds, y_train)

tensor(37.5563)

In [22]:
#what if I don't transite data type to float?
y_train = get_data()[1] # call data again
mse(preds, y_train)

TypeError: ignored

### Gradients and backward pass

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=6493)

In [0]:
def model_ping(out = 'x_train'):
    l1 = lin(x_train, w1, b1) # one linear layer
    l2 = relu(l1) # one relu layer
    l3 = lin(l2, w2, b2) # one more linear layer
    return eval(out)

In [0]:
def mse_grad(inp, targ):  #mse_input(1000,1), mse_targ (1000,1)
    # grad of loss with respect to output of previous layer
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

In [20]:
# - mse gradient
y_hat = model_ping('l3') #get value from forward model
y_hat.g = 2. *((y_hat.squeeze(-1)-y_train).unsqueeze(-1))/y_hat.shape[0]
y_hat.g.shape

torch.Size([50000, 1])

In [0]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [22]:
# - l3 gradient
lin2 = model_ping('l2'); #get value from forward model
lin2.g = y_hat.g@w2.t(); lin2.g.shape
w2.g = (lin2.unsqueeze(-1) * y_hat.g.unsqueeze(1)).sum(0);w2.g.shape
b2.g = y_hat.g.sum(0);b2.g.shape

torch.Size([50000, 50])

torch.Size([50, 1])

torch.Size([1])

In [0]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    inp.g = (inp>0).float() * out.g

In [24]:
# - relu gradient
lin1=model_ping('l1') #get value from forward model
lin1.g = (lin1>0).float() * lin2.g; lin1.g.shape

torch.Size([50000, 50])

In [25]:
# - l1 gradient
x_train.g = lin1.g @ w1.t(); x_train.g.shape
w1.g = (x_train.unsqueeze(-1) * lin1.g.unsqueeze(1)).sum(0); w1.g.shape
b1.g = lin1.g.sum(0);b1.g.shape

torch.Size([50000, 784])

torch.Size([784, 50])

torch.Size([50])

In [0]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    # we don't actually need the loss in backward!
    loss = mse(out, targ)
    
    # backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [71]:
%time forward_and_backward(x_train, y_train)

CPU times: user 3.79 s, sys: 74.9 ms, total: 3.86 s
Wall time: 1.95 s


In [0]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_train.g.clone()

In [0]:
%%script false
w1g[0,]

- We cheat a little bit and use PyTorch autograd to check our results.

In [0]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [0]:
def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # we don't actually need the loss in backward!
    return mse(out, targ)

In [75]:
%time
loss = forward(xt2, y_train)
loss.backward()

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 4.05 µs


In [76]:
%time
loss = forward(x_train, y_train)
loss.backward()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.53 µs


In [78]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig)

AssertionError: ignored

## Refactor model

### Layers as classes

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=7112)

In [0]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)-0.5
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [0]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        # Creating a giant outer product, just to sum it, is inefficient!
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [0]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [0]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [0]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model(w1, b1, w2, b2)

---

In [79]:
%time 
loss = model(x_train, y_train)
model.backward()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.96 µs


In [0]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

##### experimenting by myself(broadcasting, einsum)

- outer product -> broadcasting

In [110]:
%time tmp = x_train @ w1

CPU times: user 115 ms, sys: 871 µs, total: 116 ms
Wall time: 61.2 ms


In [111]:
tmp.shape

torch.Size([50000, 50])

In [113]:
%time
for i in range(len(x_train)):
    tmp[i] = (x_train[i,:,None] * w1).sum(0)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs


NameError: ignored

- einsum

In [95]:
lin2.shape, y_hat.shape

(torch.Size([50000, 50]), torch.Size([50000, 1]))

In [99]:
%time w2.g = (lin2.unsqueeze(-1) * y_hat.g.unsqueeze(1)).sum(0)

CPU times: user 10.7 ms, sys: 174 µs, total: 10.9 ms
Wall time: 6.92 ms


In [100]:
%time w2.g=torch.einsum("bi, bj", lin2, y_hat.g)

CPU times: user 2.14 ms, sys: 35 µs, total: 2.18 ms
Wall time: 1.77 ms


---

### Module.forward()

- Role of `__call__` changed. No more `__call__` for **implementing** forward pass.
- By initializing the forward with `__call__`, Module.forward() use overriding to maximize reusability. So any layer inherit Module, can use parent's function.

In [0]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [0]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)-0.5
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [0]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
        self.b.g = out.g.sum(0)

In [0]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [0]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [0]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [90]:
%time
loss = model(x_train, y_train)
model.backward()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.29 µs


In [0]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Without einsum

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=7484)

In [0]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ out.g
        self.b.g = out.g.sum(0)

In [0]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [152]:
%time 
loss = model(x_train, y_train)
model.backward()

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 3.81 µs


In [0]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### nn.Linear and nn.Module

In [0]:
#export
from torch import nn

In [0]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(), targ)

In [0]:
model = Model(m, nh, 1)

In [112]:
%time
loss = model(x_train, y_train)
loss.backward()

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.01 µs


## Export

In [0]:
!./notebook2script.py 02_fully_connected.ipynb

Converted 02_fully_connected.ipynb to nb_02.py
